In [ ]:
import os
import tkinter as tk
from tkinter import filedialog
import pytesseract
from PIL import Image
import shutil

# Modify these paths according to your system
path_to_tesseract = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
origine_folder_path = r'path_to_your_origine_folder'
processed_folder_path = r'path_to_your_processed_folder'

pytesseract.pytesseract.tesseract_cmd = path_to_tesseract

def create_folders():
    if not os.path.exists(origine_folder_path):
        os.makedirs(origine_folder_path)
    if not os.path.exists(processed_folder_path):
        os.makedirs(processed_folder_path)

    return origine_folder_path, processed_folder_path

def import_file():
    origine_folder_path, processed_folder_path = create_folders()
    file_path = filedialog.askopenfilename()

    if file_path:
        original_file_path, processed_file_path = process_file(file_path, origine_folder_path, processed_folder_path)
        if original_file_path and processed_file_path:
            print('File imported successfully!')
            print('Original file path:', original_file_path)
            print('Processed file path:', processed_file_path)
    else:
        print('No file selected.')

def process_file(file_path, origine_folder_path, processed_folder_path):
    file_name = os.path.basename(file_path)
    file_extension = os.path.splitext(file_name)[1].lower()

    original_file_path = os.path.join(origine_folder_path, file_name)
    shutil.copy(file_path, original_file_path)

    if file_extension in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']:
        extracted_text, processed_image_path = extract_text_from_image(original_file_path, processed_folder_path)
    else:
        extracted_text = extract_data(original_file_path)
        processed_image_path = None

    processed_text_file_path = os.path.join(processed_folder_path, file_name + '.txt')
    if extracted_text is not None:
        with open(processed_text_file_path, 'w') as processed_text_file:
            processed_text_file.write(extracted_text)

    return original_file_path, processed_text_file_path

def extract_text_from_image(image_path, processed_folder_path):
    try:
        with Image.open(image_path) as img:
            text = pytesseract.image_to_string(img)
            processed_image_path = os.path.join(processed_folder_path, os.path.basename(image_path))
            img.save(processed_image_path)
            return text, processed_image_path
    except Exception as e:
        print("Error:", e)
        return None, None

def extract_data(file_path):
    return 'Extracted data from {}'.format(file_path)

window = tk.Tk()
import_button = tk.Button(window, text='Import File', command=import_file)
import_button.pack()
window.mainloop()
